# <center> Livrable 4 : Modélisation numérique <center> Projet ESCAPE NO GAME
<center>

![image.png](attachment:image.png)
</center>


**Auteurs :** Gabriel NICOLLE | Noé POIRIER | Clément RICHARD | Antoine VON TOKARSKI  
**Groupe :** G3

# INTRODUCTION

## Contexte
Ayant réalisé la présentation de notre chaîne de transmission en expliquant en détails le fonctionnement de notre solution, nous avons du passer à l'étape supérieure, la réalisation du POC qui consiste en la création de notre chaîne de transmission via un programme python.

## Objectif
Le programme doit contenir :
- Un convertisseur en binaire
- Une création de trame (contenant un flag de départ et de fin, les données, le FSC pour détecter les erreurs)
- Une double modulation FSK - ASK
- Une double démodulation ASK - FSK
- Un convertisseur en décimal

# SOLUTION
## Programme émission - réception

### Partie émission

In [ ]:
# Importation des librairies
import sounddevice as sd
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from math import ceil

#### Le choix du type de message à envoyer

In [ ]:
# On pose une question pour demander à l'utilisateur quel type de donnée il souhaite envoyer
Access = False
while Access == False:
    choix = int(input("Entrez le type de message à envoyer\n1 : texte | 2 : sonore\n"))
    if choix == 1 or choix == 2:
        Access = True
    else:
        print("Entrée incorrecte")

#### Réception du message de l'agent (texte) et conversion en binaire

In [ ]:
if choix == 1: # Si c'est un texte
    M = input("Entrez votre message : ")
    print(M)
    # Convertir chaque caractère en sa représentation binaire ASCII sur 8 bits
    binary_representation = [format(ord(char), '08b') for char in M]

    # Regrouper tous les bits en un seul tableau
    data = np.array([list(map(int, binary_value)) for binary_value in binary_representation]).flatten()

    if (len(data) % 32 != 0):  # Si data ne peut pas être envoyé en 4 octect par 4 octects:
        while (len(data) % 32 != 0):
            data = np.append(data, [0, 0])
    puredata = data
    print(data, len(data))

#### Réception du message de l'agent (sonore) et conversion en binaire

In [ ]:
Fe = 44100 # Fréquence d'échantillonage à plus du double du maximum du spectre humain
if choix == 2: # Si c'est un son
    duree = int(input("Entrez la durée de l'enregistrement en secondes : "))
    onoff = input("Prêt ?")
    
    myrecording = sd.rec(int(duree * Fe), Fe, 1)
    print("recording...")
    sd.wait()
    print('recording finished')
    sd.play(myrecording, Fe) # Pour vérifier si l'enregistrement c'est bien passé
    sd.wait()

    t = np.linspace(0, duree, len(myrecording))
    plt.plot(t, myrecording)
    plt.xlabel("temps en $s$")
    plt.ylabel("Amplitude")
    plt.title("temps enregistré")
    plt.show()
    # On obtient des tensions entre -1 et 1. On fait +1 pour en obtenir entre 0 et 2 (on retirera 1 lors du décodage)

    myrecording_line = myrecording.ravel()
    
    valeurs_ajustes = []
    [[valeurs_ajustes.append(myrecording[T] + 1)] for T in range(0, len(myrecording))]
    
    
    def CAN(tensions, resolution, plage_de_tension):
    # Calculer la plage de valeurs possibles avec une résolution de 16 bits
        nombre_de_pos = 2 ** resolution

    # Convertir les valeurs de tension en valeurs numériques
        digital = [int((valeur / plage_de_tension) * nombre_de_pos) for valeur in tensions]

    # Passer au binaire et obtenir chaque bit individuel
        bits_liste = [int(bit) for valeur in digital for bit in format(valeur, f'0{resolution}b')]

        return bits_liste

    data = CAN(valeurs_ajustes, 16, 2)
    print("Valeurs de tension:", valeurs_ajustes[0:1000], len(valeurs_ajustes))
    print("Valeurs numériques (bit par bit):", data[0:1000], len(data), type(data))
    
    if (len(data) % 32 != 0):  # Si data ne peut pas être envoyé en 4 octect par 4 octects:
        while (len(data) % 32 != 0):
            data = np.append(data, [0, 0])

#### Création de la trame

In [ ]:
# Nous avons deux trames qui diffèrent en fonction du type de message envoyé
# La trame est la suivante :
# 1 octet flag -> 4 octets données (32 bits) -> 1 octet de correction -> 1 octet flag
# Le flag pour le texte est 1001 0011 | Le flag pour le son est 1001 1100

Message = []
flagT = [1, 0, 0, 1, 1, 1, 0, 0]
flagS = [1, 0, 0, 1, 0, 0, 1, 1]

# On fait les bits de parités pour détecter les erreurs
bits_de_p = []
for p in range(0, len(data), 4):
    bits_de_p.append(sum(data[p:p + 4]) % 2)
print('Le FSC est :', bits_de_p, len(bits_de_p))

Tau = 0
FS = 0

if choix == 1: # Si c'est un texte
    for Tau in range(0, len(data), 32):
        Message.extend(flagT)
        Message.extend(data[Tau:Tau + 32])
        Message.extend(bits_de_p[FS:FS + 8])
        FS += 8
        Message.extend(flagT)
    print("Le message est :", Message, len(Message))

if choix == 2: # Si c'est un son
    for Tau in range(0, len(data), 32):
        Message.extend(flagS)
        Message.extend(data[Tau:Tau + 32])
        Message.extend(bits_de_p[FS:FS + 8])
        FS += 8
        Message.extend(flagS)
    print("Le message est de la longueur : ", Message[0:9998], len(Message)) # Cela donne souvent trop de données à afficher donc on limite l'affichage

#### Modulation FSK

In [ ]:
# Début de la modulation et du traitement des données binaires

baud_rate = 441 # Débit en bauds (nombre de symboles par seconde)
Fe = 44100 # Fréquence d'échantillonage
Te = 1 / (baud_rate * 2) # Temps d'échantillonage

# Paramètres de la modulation
f1_FSK = 19000  # Fréquence de la porteuse 1 FSK en Hz
f2_FSK = 15000 #Fréquence de la porteuse 2 FSK en Hz
f_ASK = (f1_FSK + f2_FSK) / 2  # Fréquence de la porteuse ASK en Hz
amp_FSK = 1  # Amplitude de la porteuse utilisé pour la modulation FSK
amp_ASK = 1  # Amplitude de la porteuse utilisé pour la modulation ASK. Elle diffère de celle du FSK pour plus de simplicité plus tard

Ns = Fe / baud_rate
Message = np.repeat(Message, Ns)
duree = len(Message) * Te # Durée du message utilisé pour le vecteur temps
t = np.arange(0, duree / 2, Te)

# On choisit 3 porteuses, elles tournent toutes autour de 18 kHz
p1_FSK = amp_FSK * np.sin(2 * np.pi * f1_FSK * t) # Porteuse 1 pour le FSK
p2_FSK = amp_FSK * np.sin(2 * np.pi * f2_FSK * t) # Porteuse 2 pour le FSK
p_ASK = amp_ASK * np.sin(2 * np.pi * f_ASK * t) # Porteuse pour l'ASK

# Dans un baud, on transmet 2 bits, la modulation de fréquence modulera les bits de données
# Modulation FSK pour les bits pairs (valeur 0)
data_pair = []
for i in range(0, len(Message), 200):
    data_pair.extend(Message[i:i + 100])
data_pair = np.array(data_pair)
print(data_pair)
signal_mod_fsk1 = p1_FSK * data_pair
# Modulation FSK pour les bits pairs (valeur 0)
data_pair_inv = 1 - data_pair  # Inversion des bits impairs
signal_mod_fsk2 = p2_FSK * data_pair_inv
# On les additionne pour obtenir notre signal
print(signal_mod_fsk2)

FSK = signal_mod_fsk1 + signal_mod_fsk2

plt.plot(t[0:1000], p1_FSK[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('Porteuse FSK 1')
plt.show()

plt.plot(t[0:1000], p2_FSK[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('Porteuse FSK 2')
plt.show()

plt.plot(t[0:1000], signal_mod_fsk1[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('FSKbits1')
plt.show()

plt.plot(t[0:1000], signal_mod_fsk2[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('FSKbits0')
plt.show()
plt.plot(t[0:1000], FSK[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('FSK')
plt.show()

#### Modulation ASK

In [ ]:
# Maintenant que le FSK est fait, il nous reste la partie ASK 

# On a multiplié par 100 le nombre de bits. Pour simuler le premier bit, il faut donc prendre les 100 premiers 
# puis on décale juste le tableau en supprimant les 100 premières valeurs
data_impair = []
Message_impair = Message[100:]
print(Message_impair, len(Message_impair))
for i in range(0, len(Message), 200):
    data_impair.extend(Message_impair[i:i + 100])
data_impair = np.array(data_impair)
print(data_impair, len(data_impair))
ASK = p_ASK * data_impair

plt.plot(t[0:1000], p_ASK[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('Porteuse ASK')
plt.show()
plt.plot(t[0:1000], ASK[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('Signal modulé avec le ASK')
plt.show()

#### Addition du FSK et du ASK pour avoir notre signal modulé

In [ ]:
Message_module = FSK + ASK
plt.plot(t[0:1000], Message_module[0:1000])
plt.grid()
plt.xlabel('temps en $s$')
plt.ylabel('Amplitude')
plt.title('Signal modulé avec le ASK')
plt.show()

#### Envoi du message

In [ ]:
# On transmet le signal
print(len(Message_module))
sf.write('M_a_transmettre.WAV', Message_module, Fe)

Message_recu, Fe2 = sf.read('M_a_transmettre.WAV')
print(Message_recu, len(Message_recu))
print(Fe2)

t_transmission = len(Message_recu) / Fe
print(f"Le message est prêt à être transmit le transfert prendra exactement {t_transmission} secondes à être transmit")
countdown = input("Prêt ?")
sd.play(Message_recu,Fe)
sd.wait()

### Partie réception

In [ ]:
# On prends le texte
Message_recu, Fe2 = sf.read('M_a_transmettre.WAV')

# On capte le son provenant de l'appareil
Fe = Fe2 # fréquence d'échantillonage
onoff = input("Prêt ?")

# On connait déjà une grande majorité des paramètres
baud_rate = 441 # Débit en bauds (nombre de symboles par seconde)
Fe = 44100 # Fréquence d'échantillonage
Te = 1 / (baud_rate * 2) # Temps d'échantillonage

# Paramètres de la modulation
f1_FSK = 19000  # Fréquence de la porteuse 1 FSK en Hz
f2_FSK = 15000 #Fréquence de la porteuse 2 FSK en Hz
f_ASK = (f1_FSK + f2_FSK) / 2  # Fréquence de la porteuse ASK en Hz
amp_FSK = 1  # Amplitude de la porteuse utilisé pour la modulation FSK
amp_ASK = 1  # Amplitude de la porteuse utilisé pour la modulation ASK. Elle diffère de celle du FSK pour plus de simplicité plus tard

Ns = int(Fe / baud_rate)
duree = len(Message) * Te # Durée du message utilisé pour le vecteur temps
t = np.arange(0, duree / 2, Te)

print(len(Message_recu), Message_recu[0:1000])
plt.plot(t[0:1000], Message_recu[0:1000])
plt.xlabel('temps en $s$')
plt.ylim(-1,1)
plt.ylabel('Amplitude')
plt.title('Le message reçu brut')

#### Filtrage éventuel

In [ ]:
# Si le bruit est jugé trop important, on peut exécuter ce programme pour agir comme un filtre

fc1 = 10000 # Fréquence minimale
fc2 = 19000 # Fréquence maximale

# Méthode butter car nous avons un problème à effectuer l'inversion de la transformée de Fourier
# On conçoit un filtre passe-bande, la librairie signal possède des fonctions permettant de simuler un filtre de l'ordre de notre choix

# Fonction pour concevoir un filtre passe-bande
def butter_bandpass(lowcut, highcut, Fe2):
    nyquist = 0.5 * Fe2
    low = lowcut / nyquist
    high = highcut / nyquist
    b,a = butter(2, [low, high], btype='band', analog = False)
    return b, a

# Fonction pour appliquer le filtre passe-bande
def butter_bandpass_filter(data, lowcut, highcut, Fe2):
    b, a = butter_bandpass(fc1, fc2, Fe2)
    y = lfilter(b, a, data)
    return y

# Cette méthode nous permet d'utiliser un filtre passe-bande de l'ordre de notre choix. On commence avec un filtre d'ordre 2


record_filtered = butter_bandpass_filter(Message_recu, fc1, fc2, Fe2)
sd.play(Message_recu[0:1000], 44100)
sd.wait()
sd.play(record_filtered, 44100)
sd.wait()
plt.plot(Message_recu[0:1000])
plt.xlabel('Num échantillon')
plt.ylabel('Amplitude')
plt.title('Message brut')
plt.grid()
plt.show()
plt.plot(record_filtered)
plt.xlabel('Num échantillon')
plt.ylabel('Amplitude')
plt.title('Message filtré')
plt.grid()
plt.show()
Message_recu = record_filtered # Comme cette étape est optionelle, on ne veut pas créér une erreur au changement de nom de la variable

#### Démodulation ASK

In [ ]:
# On connait déja notre porteuse pour le ASK

# On sépare notre signal
Produit_ASK = Message_recu * p_ASK
Produit1_FSK = Message_recu * p1_FSK
Produit2_FSK = Message_recu * p2_FSK
Res = []
time_integration = np.arange(0, Ns * Te, Te)

# L'intégrale avec la méthode des trapèzes
for i in range(0, len(Message_recu)):
    plage_ask = Produit_ASK[i * Ns:(i + 1) * Ns]
    if len(plage_ask) > 0:
        Res.append(np.trapz(plage_ask, time_integration))
    else:
        # Ajouter une valeur arbitraire si la plage est vide
        Res.append(0.0)

# On peut avoir des valeurs négatives donc on prend les valeurs absolues des intégrales
for T in range(len(Res)):
    Res[T] = abs(Res[T])


# On ne garde que les intégrales non nulles
KAPP = []
for y in range(0, len(Res)):
    if Res[y] != 0:
        KAPP.append(Res[y])

# Il faudra définir un seuil à partir duquel l'amplitude donne un 1 
# On fait la moyenne des écart-types des résultats d'intégrations pour trouver notre seuil
seuil = np.mean(KAPP)
MessageF = np.zeros(len(KAPP) * 2) # On crée un tableau avec seulement des zéros, on double sa taille pour que celui-ci puisse contenir
#les informations binaires des deux démodulations
print(seuil)
ASK_effect = [] # Plus tard, pour la démodulation FSK, nous devrons retirer les effets causés par la modulation ASK pour isoler nos composantes FSK                 
for L in range(0, len(KAPP), 1):
    if KAPP[L] > seuil:
        MessageF[(2 * L) + 1] = 1 # Pour rappel, dans le programme de l'émetteur, la modulation ASK module les bits impairs
        ASK_effect.append(1)
    else:
        MessageF[(2 * L) + 1] = 0
        ASK_effect.append(0)
print(MessageF[:100])

#### Démodulation FSK

In [ ]:
ASK_effect = np.repeat(ASK_effect, Ns) # On duplique ce tableau pour plus tard inverser les changements effectués par la modulation ASK
print(ASK_effect[0:1000], len(ASK_effect))
print(Message_recu[0:1000], len(Message_recu))
for AKE in range(len(ASK_effect)):
    if ASK_effect[AKE] == 1: # Si le ASK avait modulé ce bit
        Message_recu[AKE] = Message_recu[AKE] / 2 # Pour rappel, on additionnait les deux amplitudes des porteuse ASK et FSK ayant la même amplitude.
        #Il suffit donc de diviser par 2 la valeur pour retrouver le message FSK initial
print(Message_recu[0:1000], len(Message_recu))

In [ ]:
# L'intégrale avec les trapèzes:

# On initialise les listes qui contiendront nos deux types d'intégrales
Res1 = []
Res2 = []
# L'intégrale avec FSK 1
for i in range(0, len(Message_recu)):
    plage1_FSK = Produit1_FSK[i * Ns:(i + 1) * Ns]
    if len(plage1_FSK) > 0:
        Res1.append(np.trapz(plage1_FSK, time_integration))
    else:
        break


for i in range(0, len(Message_recu)):
    plage2_FSK = Produit2_FSK[i * Ns:(i + 1) * Ns]
    if len(plage2_FSK) > 0:
        Res2.append(np.trapz(plage2_FSK, time_integration))
    else:
        break
        

# On prend les valeurs absolues de ces deux listes d'intégrales
for ab in range(len(Res1)):
    Res1[ab] = abs(Res1[ab])
    Res2[ab] = abs(Res2[ab])

print(Res1, len(Res1))
print(Res2, len(Res2))

for c in range(0, len(Res1)):
    if Res1[c] > Res2[c]:
        MessageF[2 * c] = 1 # Pour rappel, le FSK modulait les bits pairs
    else:
        MessageF[2 * c] = 0
print(MessageF[0:1000], len(MessageF))

#### Transformation du message binaire et son décodage

In [ ]:
# On réduit l'ensemble des 0 avant le message (qui commence par un 1)
# On convertit notre tableau numpy en une liste d'entiers
MessageFlist = []
for w in range(0, len(MessageF)):
    MessageFlist.append(int(MessageF[w]))

noD = 0
noF = 0

for u in range(0, len(MessageFlist) -3 ):
    if MessageFlist[u:u + 8] == [1, 0, 0, 1, 1, 1, 0, 0] or [1, 0, 0, 1, 0, 0, 1, 1]:
        break
    else:
        noD += 1
for f in range(len(MessageFlist) -3, 0):
    if MessageFlist[u -8:u] == [1, 0, 0, 1, 1, 1, 0, 0] or [1, 0, 0, 1, 0, 0, 1, 1]:
        break
    else:
        noF += 1
        
begin = noD
end = len(MessageFlist)

shorten = MessageFlist[begin:end]
print(shorten[:1000], len(shorten))

# Énonciation de notre trame.
# Flags d'entrés : 1001 1100 pour les textes et 1001 0011 pour les sons
# La différence se trouve au 5ème bit de l'octet du flag : si c'est un 1, le message est un texte | si c'est un 0, le message est un son
# On part du principe que l'ensemble du message sera ainsi soit un texte, soit un son , on ne fait donc le texte que une fois
if shorten[4] == 1:
     type_of_data = 1
     print("C'est un texte")
else:
     type_of_data = 0
     print("C'est un son")

#### Affichage du texte

In [ ]:
# Dans le cas ou c'est un texte, on a prévu la trame suivante :
# 1 octet pour le flag de début -> 4 octets de données -> 1 octet de parité (les 3 bits de parités ont déjà une place alouée) -> 2 octets pour le FCS,
# notre correction via CRC -> 1 octet pour le flag de fin

# Définition de la trame
    # flag - D = shorten[0:7]
    # address = shorten[8:23]
    # data = shorten[24:46]
    # FSC = shorten[47:62]
    # flag - F = shorten[63:71]

flagD = []
data_found = []
FSC = []
flagF = []

if type_of_data == 1: # Si c'est un texte
    Tau = 0
    for Tau in range(0, len(shorten), 56):
        if shorten[Tau:Tau + 8] == [1, 0, 0, 1, 1, 1, 0, 0]: # Si on détecte le flag
            flagD.extend(shorten[Tau:Tau + 8]) # On prend le premier octect comme flag
            data_found.extend(shorten[Tau + 8:Tau + 40])
            FSC.extend(shorten[Tau + 40:Tau + 48])
            flagF.extend(shorten[Tau + 48:Tau + 56])

    print(f"Le flag de début est {flagD}, {len(flagD)}. Il y a eu {len(flagD) / 8} paquets.")
    print(f"Les données sont {data_found}, {len(data_found)}.")
    print(f"Le FCS est {FSC}, {len(FSC)}.")
    print(f"Le flag de fin est {flagF}, {len(flagF)}.")
    print(f"Les données étaient : {shorten}.")
    print(f"{Tau} bits analysés.")


    # On vérifie l'intégrité de chaque paquet de donnée (partie data)
    error = 0
    h = 0
    for k in range(0, len(FSC)):
        if (sum((data_found[h:h + 4])) % 2) != FSC[k]:
            error += 1
        h += 4
    print(error,"erreures détectés")


    # Converstion du binaire vers ASCII:

    # Convertir en une chaîne binaire
    donnees_binaire_str = ''.join(map(str, data_found))
    print(donnees_binaire_str)

    # Convertir la chaîne binaire en ASCII
    donnees_ascii = ''.join([chr(int(donnees_binaire_str[i:i + 8], 2)) for i in range(0, len(donnees_binaire_str), 8)])

    # Afficher le résultat
    print(f"Les données en ASCII sont : {donnees_ascii}")

#### Écoute du message

In [ ]:
# Dans le cas ou c'est un texte, on a prévu la trame suivante :
# 1 octet pour le flag ->  3 octets de données mais
# les 3 bits de parités ont déjà une place alouée -> 2 octets pour le FCS, notre correction via CRC -> 1 octet pour le flag de fin
# Définition de la trame
    # flag - D = shorten[0:7]
    # address = shorten[8:23]
    # data = shorten[24:46]
    # FSC = shorten[47:62]
    # flag - F = shorten[63:71]

flagD = []
data_found = []
FSC = []
flagF = []

if type_of_data == 0: # Si c'est un son
    Tau = 0
    for Tau in range(0, len(shorten), 56):
        if shorten[Tau:Tau + 8] == [1, 0, 0, 1, 0, 0, 1, 1]:
            flagD.extend(shorten[Tau:Tau + 8]) # On prend le premier octet comme flag
            data_found.extend(shorten[Tau + 8:Tau + 40]) # Les données
            FSC.extend(shorten[Tau + 40:Tau + 48]) # Le FSC
            flagF.extend(shorten[Tau + 48:Tau + 56]) # Le flag de fin

    print(f"Le flag de début est {flagD[0:1000]}, {len(flagD)}. Il y a eu {len(flagD) / 8} paquets.")
    print(f"Les données sont {data_found[0:1000]}, {len(data_found)}.")
    print(f"Le FCS est {FSC[0:1000]}, {len(FSC)}.")
    print(f"Le flag de fin est {flagF[0:1000]}, {len(flagF)}.")
    print(f"Les données étaient : {shorten[0:1000]}.")
    print(f"{Tau} bits analysés.")
    # On vérifie l'intégrité de chaque paquet de donnée (partie data)
    error = 0
    h = 0
    for k in range(0, len(FSC)):
        if (sum((data_found[h:h + 4])) % 2) != FSC[k]:
            error += 1
        h += 4
    print(error,"erreures détectés")
    

# Conversion binaire - valeur de tension. Nos données sont en 16 bits donc il nous faut un CNA 16 bits :
    # On passe du binaire à une valeur numérique
    valeurs_numeriques = [int(''.join(map(str, data_found[i:i + 16])), 2) for i in range(0, len(data_found), 16)]

# On a des valeurs entre 0 et 2*16. Il suffit de diviser les valeurs par 2*16 et faire les opérations inverses du CAN
    Message_final = []
    for t in range(0, len(valeurs_numeriques)):
        Message_final.append(((valeurs_numeriques[t] / (2 ** 16)) - 0.5) * 2)
    
# Écoute du message
    print(Message_final[0:1000], len(Message_final))
    plt.plot(Message_final)
    plt.xlabel("num échantillon")
    plt.ylabel("Amplitude")
    sd.play(Message_final, 44100)
    sd.wait()

# CONCLUSION

En suivant notre chaîne de transmission initiale, nous avons récupéré le message fourni par l'agent afin de le coder en binaire pour le moduler en FSK et en ASK. Après avoir créé une trame afin de transmettre notre message et ainsi, l'envoyer, on a executé les étapes inverses en démodulant le signal et en le décodant afin de retrouver notre message d'origine.

En conclusion, nous avons réussi à transmettre un message en le récupérant par la suite afin de retrouver sa version initiale.